In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATASET_PATH = 'data1/'
RESAMPLE_PATH = 'resample/'

In [2]:
### Convert all the TXT files to CSV

# def txt_to_csv(txt_filename, header):
#     with open(txt_filename, 'r') as txt_file:
#         content = txt_file.readlines()

#     # Split each line into columns and remove leading/trailing whitespaces
#     content = [line.strip().split() for line in content]

#     # Write the data into a CSV file
#     csv_filename = txt_filename.replace('.txt', '.csv')
#     with open(csv_filename, 'w', newline='') as csv_file:
#         writer = csv.writer(csv_file)
#         writer.writerow(header)
#         writer.writerows(content)


In [3]:
header = ['timestamp', 'TGS2611-5.65','TGS2612-5.65','TGS2610-5.65','TGS2602-5.65','TGS2611-5.00','TGS2612-5.00','TGS2610-5.00', 'TGS2602-5.00']  # Custom header for the CSV file

# for filename in os.listdir(DATASET_PATH):
#     txt_to_csv(DATASET_PATH+filename, header)

In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor

def create_nonlinear_regression_and_resample(csv_file, output_file, time_vector, degree=5):
    # Load data from CSV file
    df = pd.read_csv(csv_file)

    # Create an empty DataFrame to store the resampled values
    resampled_df = pd.DataFrame({'timestamp': time_vector})

    # Iterate over each column and perform non-linear regression
    for col in df.columns:
        if col == 'timestamp':
            continue

        # Extract the x and y values for the regression
        x = df['timestamp'].values.reshape(-1, 1)
        y = df[col].values

        # Create polynomial features
        poly = PolynomialFeatures(degree=degree)
        x_poly = poly.fit_transform(x)

        # Create a polynomial regression model
        model = DecisionTreeRegressor()
        model.fit(x_poly, y)

        # Predict values using the fitted model for the new time vector
        time_vector_poly = poly.transform(np.array(time_vector).reshape(-1, 1))
        predicted_values = model.predict(time_vector_poly)

        # Store the predicted values in the resampled DataFrame
        resampled_df[col] = predicted_values.round(4)

    resampled_df['timestamp'] = (resampled_df['timestamp'].round(2))
    resampled_df.to_csv(output_file, index=False)

In [14]:
for filename in os.listdir(DATASET_PATH):
    input_file = DATASET_PATH + filename       # Replace 'input.csv' with your input file name
    output_file = RESAMPLE_PATH + filename     # Replace 'output.csv' with your desired output file name
    timevector = np.linspace(0, 599.99, 60000)
    
    create_nonlinear_regression_and_resample(input_file, output_file, timevector)
    break